In [51]:
import dgl
import numpy as np
import torch
import torch.nn as nn
from scipy import sparse
import networkx as nx
from scipy.sparse import diags
import torch.functional as F

In [55]:
a = np.array([1, 0, 3])
def onehot(a): 
    b = np.zeros((a.size, a.max()+1))
    b[np.arange(a.size),a] = 1
    return b
onehot(a)

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [4]:
from dgl.data import citation_graph as citegrh
data = citegrh.load_cora()

In [152]:
graph=data.graph
labels=data.labels
onehot_labels=onehot(data.labels)
train_mask=data.train_mask.astype(np.int).nonzero()[0]
train_labels = label[train_mask]

In [183]:
def load_matrix(graph):
    adj_matrix = nx.adj_matrix(graph)
    degree_vec = adj_matrix.sum(axis=1)
    with np.errstate(divide='ignore'):
        d_inv_sqrt = np.squeeze(np.asarray(np.power(degree_vec, -1)))
    d_inv_sqrt[np.isinf(d_inv_sqrt) | np.isnan(d_inv_sqrt)] = 0
    degree_matrix = diags(d_inv_sqrt, 0)
    return adj_matrix, degree_matrix

adj_matrix, degree_matrix=load_matrix(graph)

In [184]:
def lpa(adj_matrix,degree_matrix, labels,train_mask,iteration=10):
    influence=labels.copy()
    influence[np.arange(train_mask.size,labels.shape[0])]=0  # remove invisible_nodes
    for _ in range(iteration):
        influence = degree_matrix@adj_matrix@influence
        influence[train_mask]=labels[train_mask]
    pred=influence.argmax(1)
    labels=labels.argmax(1)
    acc = (pred==labels).sum()/labels.size
    return influence, acc

influence, acc =lpa(adj_matrix,degree_matrix,onehot_labels,train_mask,iteration=20)

In [185]:
acc

0.7101181683899557